In [1]:
import os
import sys
import matplotlib.pyplot as plt



PROJECT_ROOT = os.getcwd()

if PROJECT_ROOT not in sys.path:
    sys.path.insert(0, PROJECT_ROOT)

print("PROJECT_ROOT:", PROJECT_ROOT)
print("Contenido de ./data:", os.listdir(os.path.join(PROJECT_ROOT, "data")))

PROJECT_ROOT: c:\Users\adria.flores\Documents\Projects\hacks\datathon2025-smadex
Contenido de ./data: ['sample_submission.csv', 'test', 'train']


In [2]:
import dask
import dask.dataframe as dd
# When dask uses pandas 2.0.0+ it casts object columns to string automatically ([("i", 0.48)] -> '[("i", 0.48)]')
dask.config.set({"dataframe.convert-string": False})

dataset_path = "./data/train"
filters = [("datetime", ">=", "2025-10-01-00-00"), ("datetime", "<", "2025-10-13-00-00")]

ddf = dd.read_parquet(
    dataset_path,
    filters=filters
)

In [ ]:
# 1. Basic structure
print("=== Basic info ===")
print("Number of partitions:", ddf.npartitions)
print("Columns:", list(ddf.columns))

print("\n=== Dtypes ===")
print(ddf.dtypes)

# 2. Quick look at the data
print("\n=== Head ===")
display(ddf.head())  # small preview, does not require full compute

# 3. Summary statistics for numeric columns
print("\n=== Numeric summary (describe) ===")
numeric_summary = ddf.describe().compute()
display(numeric_summary)

# 4. Missing values per column (as percentage)
print("\n=== Missing values (%) per column ===")
missing_pct = (ddf.isna().mean() * 100).compute()
display(missing_pct.sort_values(ascending=False))

# 5. Categorical / object-like columns: top values
print("\n=== Top values for categorical columns ===")
cat_cols = ddf.select_dtypes(include=["object", "category"]).columns

for col in cat_cols:
    print(f"\nColumn: {col}")
    vc = ddf[col].value_counts().compute().head(10)
    display(vc)

# 6. Numeric distributions: histograms for a subset of columns
print("\n=== Histograms for numeric columns (sample) ===")
numeric_cols = ddf.select_dtypes(include=["number"]).columns

# Take a sample to avoid computing the entire dataset
sample = ddf[numeric_cols].sample(frac=0.05, random_state=42).compute()

for col in numeric_cols:
    plt.figure()
    sample[col].hist(bins=30)
    plt.title(f"Histogram of {col}")
    plt.xlabel(col)
    plt.ylabel("Frequency")
    plt.show()


=== Basic info ===
Number of partitions: 144
Columns: ['buyer_d1', 'buyer_d7', 'buyer_d14', 'buyer_d28', 'buy_d7', 'buy_d14', 'buy_d28', 'iap_revenue_d7', 'iap_revenue_d14', 'iap_revenue_d28', 'registration', 'retention_d1_to_d7', 'retention_d3_to_d7', 'retention_d7_to_d14', 'retention_d1', 'retention_d3', 'retentiond7', 'advertiser_bundle', 'advertiser_category', 'advertiser_subcategory', 'advertiser_bottom_taxonomy_level', 'carrier', 'country', 'region', 'dev_make', 'dev_model', 'dev_os', 'dev_osv', 'hour', 'release_date', 'release_msrp', 'weekday', 'avg_act_days', 'avg_daily_sessions', 'avg_days_ins', 'avg_duration', 'bcat', 'bcat_bottom_taxonomy', 'bundles_cat', 'bundles_cat_bottom_taxonomy', 'bundles_ins', 'city_hist', 'country_hist', 'cpm', 'cpm_pct_rk', 'ctr', 'ctr_pct_rk', 'dev_language_hist', 'dev_osv_hist', 'first_request_ts', 'first_request_ts_bundle', 'first_request_ts_category_bottom_taxonomy', 'hour_ratio', 'iap_revenue_usd_bundle', 'iap_revenue_usd_category', 'iap_revenu

,buyer_d1,buyer_d7,buyer_d14,buyer_d28,buy_d7,buy_d14,buy_d28,iap_revenue_d7,iap_revenue_d14,iap_revenue_d28,...,user_bundles_l28d,weekend_ratio,weeks_since_first_seen,wifi_ratio,whale_users_bundle_num_buys_prank,whale_users_bundle_revenue_prank,whale_users_bundle_total_num_buys,whale_users_bundle_total_revenue,row_id,datetime
0,0,1,1,1,1,1,1,2.147718,2.147718,2.147718,...,"[88981729bd5c1e5aea9ada4bce00a2531e9e98f7, 25c...",0.019802,6.0,0.913366,None,None,None,None,819ecc0e-1a97-43ed-83f6-b9ede4f7fc48,2025-10-01-00-00
1,0,0,0,0,0,0,0,0.000000,0.000000,0.000000,...,None,NaN,NaN,NaN,None,None,None,None,0a7fbf18-5041-42af-bd0a-0cb6586b8598,2025-10-01-00-00
2,0,0,0,0,0,0,0,0.000000,0.000000,0.000000,...,"[6506b7e0a24666debd08f74266800f2eb154df5a, 150...",0.399021,6.0,0.999388,None,None,None,None,fc1a2689-b136-4ffa-b23b-9d8215bd720f,2025-10-01-00-00
3,0,0,0,0,0,0,0,0.000000,0.000000,0.000000,...,"[2b472e3dc96f1847490d7411b25e12ed417b9714, 3ba...",0.121547,6.0,1.000000,None,None,None,None,0340fcc6-50bd-42ab-b9f4-4c1184b640cb,2025-10-01-00-00
4,0,0,0,0,0,0,0,0.000000,0.000000,0.000000,...,"[1031535cf2a1315422fd05d321349bcd3c3ffc04, 478...",0.293285,6.0,0.160243,None,None,None,None,219d253f-bef4-4039-84b2-ed55f009cc43,2025-10-01-00-00



=== Numeric summary (describe) ===


,buyer_d1,buyer_d7,buyer_d14,buyer_d28,buy_d7,buy_d14,buy_d28,iap_revenue_d7,iap_revenue_d14,iap_revenue_d28,...,release_msrp,weekday,avg_act_days,avg_days_ins,first_request_ts,last_buy,last_ins,weekend_ratio,weeks_since_first_seen,wifi_ratio
count,2.060058e+07,2.060058e+07,2.060058e+07,2.060058e+07,2.060058e+07,2.060058e+07,2.060058e+07,2.060058e+07,2.060058e+07,2.060058e+07,...,1.847903e+07,2.060058e+07,1.117271e+07,921996.000000,9.405562e+06,3.776750e+05,3.026575e+06,1.160027e+07,1.224896e+07,1.159948e+07
mean,2.376821e-02,3.343634e-02,3.532721e-02,3.717182e-02,7.203491e-02,9.582308e-02,1.322014e-01,1.437688e+00,1.773321e+00,2.332586e+00,...,5.225308e+02,4.389697e+00,3.862084e+00,5.392996,1.758974e+09,1.758342e+09,1.758553e+09,3.316598e-01,4.882932e+00,5.782966e-01
std,1.523263e-01,1.797731e-01,1.846055e-01,1.891827e-01,9.459988e-01,1.524582e+00,2.566221e+00,4.312014e+02,4.415945e+02,4.824463e+02,...,4.969947e+02,1.968518e+00,2.085652e+00,5.666884,4.459769e+05,7.676716e+05,7.536146e+05,2.719729e-01,1.940092e+00,4.176631e-01
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,-5.800000e+00,-1.430459e+01,...,2.300000e+01,1.000000e+00,1.000000e+00,0.000000,1.758067e+09,1.756598e+09,1.756771e+09,0.000000e+00,0.000000e+00,0.000000e+00
25%,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,1.820000e+02,3.000000e+00,2.000000e+00,1.000000,1.758499e+09,1.758367e+09,1.758277e+09,1.470588e-01,5.000000e+00,2.692308e-01
50%,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,3.000000e+02,5.000000e+00,4.000000e+00,4.000000,1.759190e+09,1.758870e+09,1.758954e+09,2.997022e-01,6.000000e+00,8.531469e-01
75%,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,8.990000e+02,6.000000e+00,6.000000e+00,8.500000,1.759363e+09,1.759104e+09,1.759363e+09,5.000000e-01,6.000000e+00,1.000000e+00
max,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,5.390000e+02,1.324000e+03,2.443000e+03,8.611911e+05,8.611911e+05,8.611911e+05,...,5.160000e+03,7.000000e+00,7.000000e+00,28.000000,1.759622e+09,1.759709e+09,1.759709e+09,1.000000e+00,6.000000e+00,1.000000e+00



=== Missing values (%) per column ===
